In [119]:
import sys
sys.path.append('D:/source/repos')
from utilities.std_imports import *
pd.set_option('display.max_columns', None)
from itertools import accumulate
from scipy.optimize import linprog

In [120]:
horizon = 5
on_hand = 370
dmd = np.array([0, 130, 160, 120, 260, 130])
plan = pd.DataFrame(columns=list(range(horizon+1)))
plan.loc[0] = dmd
plan.loc[1] = np.array([0, 60, 80, 0, 100, 0])
stk = np.zeros(horizon+1)
stk[0] = on_hand
plan.loc[2] = stk
plan.index = ['Dmd', 'Pl.Or', 'Stock']
plan

,0,1,2,3,4,5
Dmd,0,130,160,120,260,130
Pl.Or,0,60,80,0,100,0
Stock,370,0,0,0,0,0


In [121]:
for i in range(1, horizon+1):
    plan.loc['Stock'][i] = plan.loc['Stock'][i-1] - plan.loc['Dmd'][i] + plan.loc['Pl.Or'][i]
plan

,0,1,2,3,4,5
Dmd,0,130,160,120,260,130
Pl.Or,0,60,80,0,100,0
Stock,370,300,220,100,-60,-190


In [110]:
plan.loc['Pl.Or'][0] = on_hand 
acStkPo = np.cumsum(plan.loc['Pl.Or'])
acDmd = np.cumsum(plan.loc['Dmd'])
plan.loc['AcStock'] = acStkPo - acDmd 
plan

,0,1,2,3,4,5
Dmd,0,130,160,120,260,130
Pl.Or,370,60,80,0,100,0
Stock,370,300,220,100,-60,-190
AcStock,370,300,220,100,-60,-190


In [95]:
(plan.loc['Pl.Or']/(plan.loc['Pl.Or'] + 0.001)).sum()

6.999980891570823

In [96]:
e = 0.001
plan.loc['Pl.Or'].sum()

2585

$ s_c = [s_0 - h.d_1 - (h-1)d_2  - (h-2)d_3  - (h-3)d_4  - (h-4)d_5 + h.p_1 + (h-1)p_2  + (h-2)p_3  + (h-3)p_4  + (h-4)p_5] . s_{uc} = s_{uc}. [s_0 - \sum_{i=0}^h (h-i) d_{i+1} + \sum_{i=0}^h (h-i) p_{i+1} ]$

$ f_c = [p_1/(p_1 + e) + p_2)/(p_2) + e))  + p_3/(p_3 + e) + p_4/(p_4 + e)  + p_5/(p_5 + e)] . f_{uc} = f_{uc}.\sum_{i=0}^h p_i/(p_i + e) $

 $ v_c = [h.p_1 + (h-1)p_2  + (h-2)p_3  + (h-3)p_4  + (h-4)p_5] . v_{uc} = v_{uc}.\sum_{i=0}^h (h-i) p_{i+1}$

$ s_c + f_c + v_c$

$ p_o - 𝑑_1 + p_1 \ge 0 $  
$ p_o - 𝑑_1 - d_2 + p_1 + p_2 \ge 0 $  
$ s_o - 𝑑_1 - d_2 - d_3 + p_1 + p_2 + p_3 \ge 0 $  
$ s_o - 𝑑_1 - d_2 - d_3 - d_4 + p_1 + p_2 + p_3 + p_4 \ge 0 $  
$ s_o - 𝑑_1 - d_2 - d_3 - d_4 - d_5 + p_1 + p_2 + p_3 + p_4 + p_5 \ge 0 $  

In [126]:
h=20
so=370
suc = 0.8
fuc = 0.3
vuc = 0.02
d = acDmd
p = acStkPo

c = [h*suc, (h-1)*suc, (h-2)*suc, (h-3)*suc, (h-4)*suc, h*vuc, (h-1)*vuc, (h-2)*vuc, (h-3)*vuc, (h-4)*vuc]

In1 = p[0] - d[1] + p[1] >= 0
In2 = p[0] - 𝑑[1] - d[2] + p[1] + p[2] >= 0     
In3 = p[0] - 𝑑[1] - d[2] - d[3] + p[1] + p[2] + p[3] >= 0     
In4 = p[0] - 𝑑[1] - d[2] - d[3] - d[4] + p[1] + p[2] + p[3] + p[4] >= 0  
In5 = p[0] - 𝑑[1] - d[2] - d[3] - d[4] - d[5] + p[1] + p[2] + p[3] + p[4]+ p[5] >= 0  

bounds = [(0, float("inf")),  (0, float("inf")), ,  (0, float("inf")), ,  (0, float("inf")), ,  (0, float("inf"))] 